In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pathlib import Path

In [9]:
# create path
staten_island = Path('Resources/staten_island.xlsx')

# read file
SI = pd.read_excel(staten_island)

SI.head()

,OBJECTID,Complaint ID,Offense,Latitude,Longitude,Borough,Public Area
0,1,261214832,ROBBERY,40.643566,-74.074001,NaN,NaN
1,2,261233859,ROBBERY,40.635961,-74.161365,NaN,NaN
2,3,261233841,ROBBERY,40.643569,-74.098338,NaN,NaN
3,4,261261343,ROBBERY,40.630408,-74.142977,NaN,NaN
4,5,261347872,ROBBERY,40.630539,-74.076641,NaN,NaN
